<a href="https://colab.research.google.com/github/Ojochideee/Portfolio/blob/main/Data_acquisition_Topp100books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Designing and Developing a Data Acquisition and Preprocessing Pipeline.

Data Acquistion: refers to the process of collecting, recording, and digitizing information from various sources such as sensors, instruments, or other devices. It's a fundamental step in many scientific, industrial, and engineering applications where data is needed for analysis, monitoring, and control.

Preprocessing Pipleline: refers to a series of data preprocessing steps applied to raw data before it is used for analysis or modeling


For my project focusing on data acquisition and preprocessing, I've opted to explore Goodreads, a website renowned as an American social cataloging platform. Operating as a subsidiary of Amazon, Goodreads enables users to delve into its extensive database encompassing books, annotations, quotes, and reviews. the specific page for this was this "Best 100 Books ever of all times " https://www.goodreads.com/list/show/1.Best_Books_Ever
The features extracted are as follows:
1. Number (position on the list)
2. Author Name
3. Book Title
4. Average Rating
5. Ratings
6. Score
7. Votes               
8. Rating Category     
9. Publishing Date        
10. Pages (no of pages per book)                
11. Rating per Vote Ratio  
12. Score per Rating       
13. Publishing Year



In [1]:
import requests # The requests module enables us to send HTTP requests easily and to interact with web servers and retrieve data from URLs, such as HTML content, JSON data, or files.
import pandas as pd # Pandas is a data manipulation and analysis library in Python.
from bs4 import BeautifulSoup # This is Python library for parsing HTML and XML documents.

In [2]:
# Creating a variable to store the url of the webpage 'goodreads'. Goodreads is a popular social cataloging website that allows individuals to search for, rate, review, and discuss books.
# for this particular webpage 'best books ever' is a list of 100 books that have been rated the best books ever, it is interactive so whenever a book is rated higher it falls into the number it is rated as.
bestbook_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever"

# set a custom user agent header in my request in order to mimic a real browser user agent to avoid detection because the server refuses to authorize it
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

In [3]:
# with the response <Response [200]> indicates that the request to retrieve the webpage was successful.
page = requests.get(bestbook_url,headers=headers)

Web Scraping with Beautiful soup

In [4]:
soup = BeautifulSoup(page.text, 'html') # To navigate through the HTML structure and extract specific features for our dataset

In [5]:
table = soup.find('table',class_ = 'tableList js-dataTooltip') # method to locate a <table> element with the class tableList

Extraction of features and Transformations as appropriate.

In [6]:
#creating a list to store the numbers 1 - 100 from the best book webpage
numbers = []
for number in table.find_all('td', {'class': "number"}):
  numbers.append(int(number.text))

In [7]:
# Extracting the authors names
authors = []
for name in table.find_all('a',{'class': "authorName"}):
  authors.append(name.text.strip())

In [8]:
# Extracting the Book Titles
bookTitle = []
for book in table.find_all('a',{'class' : "bookTitle"}):
  bookTitle.append(book.text.strip())

In [9]:
# Extracting the Average ratings
import re # The re module in Python provides support for regular expressions

avrRatings = []
for rating in table.find_all('span', class_="minirating"):
    rating = rating.text.replace(' avg rating', '').replace(' ratings', '')
    # Use regular expression to extract numerical part, in order to find float numbers in a string.
    rating_value = re.findall(r'\d+\.\d+', rating)
    if rating_value:
        avrRatings.append(float(rating_value[0]))

In [10]:
# Extracting the number of ratings per book
ratings = []
for rating in table.find_all('span', class_="minirating"):
    rating_text = rating.text.replace(' avg rating', '').replace(' ratings', '')
    # Split the rating_text using '—' as a delimiter and take only the second part, used to clean up any extra whitespace around it.
    rating_value = rating_text.split('—')[1].strip()
    ratings.append(float(rating_value.replace(',', '')))

In [11]:
# Extracting the scores per rating of the books
scores = []
for score in table.find_all('a', string=lambda text: text and 'score:' in text.lower()):
  score = score.text.strip().split(': ')[-1]
  score = int(score.replace(',', ''))
  scores.append(score)

The function 'lambda' checks if the text content of each element (text) is not empty and if the substring 'score:' (converted to lowercase) is present in the text content. If both conditions are true, the element is included in the results

In [12]:
# Extracting votes
peopleVotes = []
for vote in table.find_all('a', string=lambda text: text and 'people voted' in text.lower()):
    vote_text = vote.text.strip().split(': ')[-1]  # Extract the vote count part after ': '
    vote_count_text = ''.join(filter(str.isdigit, vote_text))  # Extract only numerical characters
    vote_count = int(vote_count_text) if vote_count_text else 0  # Convert to integer, default to 0 if empty
    peopleVotes.append(vote_count)

Creating a dictionary to store the date of the features

In [13]:
# Creating a dictionary to store the list of all the features extracted
data = {
    'Number': numbers,
    'Author name': authors,
    'Book Title': bookTitle,
    'Average Rating': avrRatings,
    'Ratings':ratings,
    'Score':scores,
    'Votes':peopleVotes
}

In [14]:
# Creating the Best books Dataframe using pandas (pd)
df = pd.DataFrame(data)
df

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,8741310.0,3805748,38719
1,2,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.50,3409636.0,3104635,31717
2,3,Jane Austen,Pride and Prejudice,4.29,4299888.0,2704983,27746
3,4,Harper Lee,To Kill a Mockingbird,4.26,6175757.0,2430034,24816
4,5,Markus Zusak,The Book Thief,4.39,2580717.0,1809817,18607
...,...,...,...,...,...,...,...
95,96,Edgar Allan Poe,The Complete Stories and Poems,4.39,250629.0,300779,3400
96,97,Anne Rice,Interview with the Vampire (The Vampire Chroni...,4.02,592834.0,295990,3307
97,98,Miguel de Cervantes Saavedra,Don Quixote,3.90,273428.0,293469,3329
98,99,Ernest Hemingway,The Old Man and the Sea,3.80,1158821.0,288767,3310


Data Transformation:
Binning and Bucketing, i.e transforming numerical features into categorical features, using a set of thresholds.
Creating a new column to transform the 'AVERAGE RATING' into discrete intervals 'Low','medium' and 'high'.

In [15]:
bins_category = [0,3,4,5]
bin_labels = ['Low', 'Medium', 'High']
df['Rating Category'] = pd.cut(df['Average Rating'], bins=bins_category, labels=bin_labels, include_lowest=True)
df

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes,Rating Category
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,8741310.0,3805748,38719,High
1,2,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.50,3409636.0,3104635,31717,High
2,3,Jane Austen,Pride and Prejudice,4.29,4299888.0,2704983,27746,High
3,4,Harper Lee,To Kill a Mockingbird,4.26,6175757.0,2430034,24816,High
4,5,Markus Zusak,The Book Thief,4.39,2580717.0,1809817,18607,High
...,...,...,...,...,...,...,...,...
95,96,Edgar Allan Poe,The Complete Stories and Poems,4.39,250629.0,300779,3400,High
96,97,Anne Rice,Interview with the Vampire (The Vampire Chroni...,4.02,592834.0,295990,3307,High
97,98,Miguel de Cervantes Saavedra,Don Quixote,3.90,273428.0,293469,3329,Medium
98,99,Ernest Hemingway,The Old Man and the Sea,3.80,1158821.0,288767,3310,Medium


Retriving date of publications involved extracting all individual urls to find the date of publications for each book.
firstly to avoid error, i stored the urls in a text file, to make it easier to manage and update them.

In [16]:
with open('/content/urls.txt', 'r') as file:
    book_urls = file.readlines()
book_urls

['https://www.goodreads.com/book/show/2767052-the-hunger-games\n',
 'https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix\n',
 'https://www.goodreads.com/book/show/1885.Pride_and_Prejudice\n',
 'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird\n',
 'https://www.goodreads.com/book/show/19063.The_Book_Thief\n',
 'https://www.goodreads.com/book/show/41865.Twilight\n',
 'https://www.goodreads.com/book/show/170448.Animal_Farm\n',
 'https://www.goodreads.com/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set\n',
 'https://www.goodreads.com/book/show/11127.The_Chronicles_of_Narnia\n',
 'https://www.goodreads.com/book/show/11870085-the-fault-in-our-stars\n',
 'https://www.goodreads.com/book/show/5297.The_Picture_of_Dorian_Gray\n',
 'https://www.goodreads.com/book/show/370493.The_Giving_Tree\n',
 'https://www.goodreads.com/book/show/18405.Gone_with_the_Wind\n',
 'https://www.goodreads.com/book/show/6185.Wuthering_Heights\n',
 'https://www.goodreads.com/book/sho

In [17]:
# Iterating over each URL in the book_urls list, and then creating a header to mimic a web browser, helping to avoid potential issues
book_urls = [url.strip() for url in book_urls]

In [18]:
# Creating a list and then extracting the publication dates of each book
publishing_dates = []

# Loop through each URL in book_urls
for url in book_urls:
    # Sending a GET request to the URL
    url_response = requests.get(url, headers=headers)

    # Checking if the request was successful
    if url_response.status_code == 200:

        url_soup = BeautifulSoup(url_response.content, 'html.parser')
        elems = url_soup.find_all('div', class_='FeaturedDetails')

        # Defining a pattern to match the date format
        date_pattern = re.compile(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}')

        for elem in elems:
          text = elem.get_text(strip=True)  # removing any leading/trailing whitespace
          # Searching for the pattern in the text
          match = date_pattern.search(text)
          if match:
            # If a match is found, append the matched date to the list
            publishing_dates.append(match.group())

    else: #else statement prints specific url that fails to be fetched
        print(f"Failed to fetch URL: {url}")


In [19]:
#After iterating throught the urls, i realized that each time there is always one random missing date for a book, so inorder to avoid that i had to fill missing dates with a placeholder value 'None'
while len(publishing_dates) < len(df):
    publishing_dates.append(None)

df['Publishing Date'] = publishing_dates


In [20]:
page_counts = []

for url in book_urls:
    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        page_soup = BeautifulSoup(response.content, 'html.parser')

        # Find the FeaturedDetails div
        featured_details = page_soup.find('div', class_='FeaturedDetails')

        # Extract page count
        if featured_details:
            # Find the <p> tag containing the page count
            page_count_elem = featured_details.find('p', {'data-testid': 'pagesFormat'})

            # Extract page count if the element is found
            if page_count_elem:
                page_count_text = page_count_elem.get_text(strip=True)
                # Use regular expression to extract the integer part
                page_count_match = re.search(r'(\d+)', page_count_text)
                if page_count_match:
                    page_count = int(page_count_match.group(1))
                    page_counts.append(page_count)
    else:
        print(f"Failed to fetch URL: {url}")

In [21]:
#looping through the page counts and then assigning 'none' to fill in ang missing value.
while len(page_counts) < len(df):
    page_counts.append(None)
df['Pages'] = page_counts

Converting the 'Publishing dates' into datetime format in python

In [22]:
#df['Publishing Date'] = pd.to_datetime(df['Publishing Date'])
# the above code brought error because there were some missing values which couldnt be converted to a date time format
#The errors parameter is set to 'coerce', which means that if some values are not in a valid date format, those will be set to NaT (Not a Time) in the output.
df['Publishing Date'] = pd.to_datetime(df['Publishing Date'],errors='coerce')

In [23]:
# Removing the 'Time Part' from the date because it isn't relevant to to the dataset
df['Publishing Date'] = df['Publishing Date'].dt.date

In [24]:
# fill missing values with (NaNs), ensuring that there are no missing values in the column
df['Publishing Date'] = df['Publishing Date'].fillna(method='ffill')

Handling missing or duplicated data from the dataset

In [25]:
df.duplicated().sum() #checking to find sum of duplicated data

0

In [26]:
df.isnull().sum() #checking the sum of all missing data

Number             0
Author name        0
Book Title         0
Average Rating     0
Ratings            0
Score              0
Votes              0
Rating Category    0
Publishing Date    0
Pages              0
dtype: int64

In [27]:
#in order to avoid any missing data, i opted to fil the missing data with the previous values on the dataset, this will help to keep the total values to '100'
df['Publishing Date'] = df['Publishing Date'].fillna(method='ffill')
df['Pages'] = df['Pages'].fillna(method='ffill')

In [28]:
# checking the final information about the dataset and the features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Number           100 non-null    int64   
 1   Author name      100 non-null    object  
 2   Book Title       100 non-null    object  
 3   Average Rating   100 non-null    float64 
 4   Ratings          100 non-null    float64 
 5   Score            100 non-null    int64   
 6   Votes            100 non-null    int64   
 7   Rating Category  100 non-null    category
 8   Publishing Date  100 non-null    object  
 9   Pages            100 non-null    int64   
dtypes: category(1), float64(2), int64(4), object(3)
memory usage: 7.4+ KB


Extracting other features from relevant features in the dataset

In [29]:
#Creating Rating per Vote Ratio: Calculating the ratio of ratings to votes.
df['Rating per Vote Ratio'] = df['Ratings'] / df['Votes']

In [30]:
#Score per Rating Ratio: Calculating the ratio of the 'Score' to 'Ratings'. i.e by dividing the 'Ratings' for the 'Score' features
df['Score per Rating'] = df['Score'] / df['Ratings']

In [31]:
#Extract the year from the 'Publishing Date' column to find the year of publications
df['Publishing Year'] = pd.to_datetime(df['Publishing Date']).dt.year

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Number                 100 non-null    int64   
 1   Author name            100 non-null    object  
 2   Book Title             100 non-null    object  
 3   Average Rating         100 non-null    float64 
 4   Ratings                100 non-null    float64 
 5   Score                  100 non-null    int64   
 6   Votes                  100 non-null    int64   
 7   Rating Category        100 non-null    category
 8   Publishing Date        100 non-null    object  
 9   Pages                  100 non-null    int64   
 10  Rating per Vote Ratio  100 non-null    float64 
 11  Score per Rating       100 non-null    float64 
 12  Publishing Year        100 non-null    int32   
dtypes: category(1), float64(4), int32(1), int64(4), object(3)
memory usage: 9.3+ KB


Descriptive Statistics of the numeric columns in the dataset,  It computes various summary statistics such as count, mean, standard deviation, minimum, maximum, and quartile information

In [33]:
df.describe()

,Number,Average Rating,Ratings,Score,Votes,Pages,Rating per Vote Ratio,Score per Rating,Publishing Year
count,100.000000,100.000000,1.000000e+02,1.000000e+02,100.000000,100.00000,100.000000,100.000000,100.000000
mean,50.500000,4.148800,2.001905e+06,7.687886e+05,8143.530000,446.77000,273.592759,0.614286,1955.940000
std,29.011492,0.206912,1.746691e+06,5.764667e+05,5819.287249,291.49371,253.254581,1.111741,49.387526
min,1.000000,3.650000,1.342870e+05,2.878720e+05,3163.000000,38.00000,8.677113,0.051462,1813.000000
25%,25.750000,3.990000,8.555765e+05,4.082832e+05,4553.500000,271.00000,156.927836,0.298005,1938.750000
50%,50.500000,4.150000,1.384811e+06,5.924680e+05,6503.000000,373.00000,208.690367,0.441217,1963.500000
75%,75.250000,4.300000,2.520539e+06,9.109172e+05,9530.250000,497.25000,311.945904,0.601065,1997.000000
max,100.000000,4.620000,1.014058e+07,3.805748e+06,38719.000000,1728.00000,1799.987467,11.110398,2012.000000


In [34]:
df[df['Ratings']>4.5] #All ratings in the webpage were greater than 4.5, as per why it is called the best books.

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes,Rating Category,Publishing Date,Pages,Rating per Vote Ratio,Score per Rating,Publishing Year
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,8741310.0,3805748,38719,High,2008-09-14,374,225.762804,0.435375,2008
1,2,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.50,3409636.0,3104635,31717,High,2003-06-21,912,107.501844,0.910547,2003
2,3,Jane Austen,Pride and Prejudice,4.29,4299888.0,2704983,27746,High,1813-01-28,279,154.973257,0.629082,1813
3,4,Harper Lee,To Kill a Mockingbird,4.26,6175757.0,2430034,24816,High,1960-07-11,323,248.861904,0.393480,1960
4,5,Markus Zusak,The Book Thief,4.39,2580717.0,1809817,18607,High,2005-09-01,592,138.696028,0.701285,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Edgar Allan Poe,The Complete Stories and Poems,4.39,250629.0,300779,3400,High,1976-04-12,821,73.714412,1.200097,1976
96,97,Anne Rice,Interview with the Vampire (The Vampire Chroni...,4.02,592834.0,295990,3307,High,1976-04-12,346,179.266405,0.499280,1976
97,98,Miguel de Cervantes Saavedra,Don Quixote,3.90,273428.0,293469,3329,Medium,1998-09-24,1023,82.135176,1.073295,1998
98,99,Ernest Hemingway,The Old Man and the Sea,3.80,1158821.0,288767,3310,Medium,1952-09-01,546,350.096979,0.249190,1952


In [35]:
df[df['Author name']=='J.K. Rowling'] # checking to see how many books written by the author 'J.K. Rowling', found in the list

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes,Rating Category,Publishing Date,Pages,Rating per Vote Ratio,Score per Rating,Publishing Year
1,2,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.50,3409636.0,3104635,31717,High,2003-06-21,912,107.501844,0.910547,2003
50,51,J.K. Rowling,Harry Potter and the Deathly Hallows (Harry Po...,4.62,3736288.0,586325,6331,High,2007-07-21,260,590.157637,0.156927,2007
53,54,J.K. Rowling,Harry Potter and the Sorcerer’s Stone (Harry P...,4.47,10140579.0,577185,6159,High,1993-04-26,333,1646.465173,0.056918,1993
64,65,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.58,4228470.0,509776,5567,High,1999-07-08,275,759.559907,0.120558,1999


In [36]:
df[df['Author name']=='William Shakespeare']

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes,Rating Category,Publishing Date,Pages,Rating per Vote Ratio,Score per Rating,Publishing Year
28,29,William Shakespeare,Romeo and Juliet,3.74,2622372.0,874474,9366,Medium,1908-01-01,320,279.988469,0.333467,1908


In [37]:
df[df['Author name']=='Charles Dickens']

,Number,Author name,Book Title,Average Rating,Ratings,Score,Votes,Rating Category,Publishing Date,Pages,Rating per Vote Ratio,Score per Rating,Publishing Year
61,62,Charles Dickens,A Tale of Two Cities,3.87,951825.0,533310,5848,Medium,1955-09-01,489,162.760773,0.560303,1955
76,77,Charles Dickens,Great Expectations,3.79,807465.0,401708,4603,Medium,1991-06-01,544,175.421464,0.497493,1991
